In [48]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [49]:
def recommendation(csv_file): #valuation,location,sq_feet etc.
    df=pd.read_csv(csv_file+".csv",index_col='Users')
    return df
df=recommendation("valuation")

In [50]:
df=df.fillna(0)

In [51]:
def convert_int(df):
    for i in df.columns:
        df[i]=df[i].astype('int')
    return df
df=convert_int(df)

In [52]:
def calculate_percentages(df):
    total=0
    for i in df.columns:
        total=df[i]+total
    df = df.pipe(lambda x: (x*100).div(total, axis='index'))
    df=df.fillna(0)
    return df
df=calculate_percentages(df)

In [54]:
def standardize(row):
    new_row = (row - row.mean())/(row.max()-row.min())
    return new_row

df_std = df.apply(standardize).T
def correlation(df_std,df):

    sparse_df = sparse.csr_matrix(df_std.values)
    corrMatrix = pd.DataFrame(cosine_similarity(sparse_df),index=df.columns,columns=df.columns)
    return corrMatrix
corrMatrix=correlation(df_std,df)

In [66]:
def get_similar(warehouse,percentage):
    similar_score = corrMatrix[warehouse]*(percentage-50.5) #50..5 is the avg of percentage (0-100)
    similar_score = similar_score.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_score
similar_score=get_similar("bluechip",2)

In [72]:
def final_recommendation(portfolio):

    similar_scores = pd.DataFrame()
    for warehouse,percentages in portfolio:
        similar_scores = similar_scores.append(get_similar(warehouse,percentages),ignore_index = True)

    similar_scores.head(10)
    return similar_scores

portfolio = [("bluechip",2.038503),("large_cap",0),("mid_cap",0)]

similar_scores=final_recommendation(portfolio)

/var/folders/yg/rsw8j92510j7cjvvkgjr4s2r0000gn/T/ipykernel_6050/2250151009.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_scores = similar_scores.append(get_similar(warehouse,percentages),ignore_index = True)


In [73]:
similar_scores.sum().sort_values(ascending=False)[:2]

small_cap    55.818374
mid_cap     -20.426789
dtype: float64